https://quantdare.com/concepts-of-entropy-in-finance-transfer-entropy/

In [1]:
import polars as pl
import polars_ds as pld
import numpy as np

In [2]:
import yfinance as yf
import pandas as pd

In [3]:
VIX = yf.Ticker('^VIX')
SP500 = yf.Ticker('^GSPC')
vix = pl.DataFrame(VIX.history(start="2020-01-01").to_dict('list'))
sp500 = pl.DataFrame(SP500.history(start="2020-01-01").to_dict('list'))


/Users/remiadon/polars_ds_extension/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
/Users/remiadon/polars_ds_extension/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [4]:
log_return = pl.col('Close').truediv(pl.col('Close').shift(1)).log().slice(1, pl.len())
df = pl.concat([vix.select(log_return.alias('log_return_vix')), sp500.select(log_return.alias('log_return_sp500'))], how="horizontal")
df.head()

log_return_vix,log_return_sp500
f64,f64
0.117159,-0.007085
-0.0122,0.003527
-0.004342,-0.002807
-0.024965,0.00489
-0.070056,0.006633


Checking `entknn` from `pycopent` gives similar results as our implemenation 

In [5]:
from copent import entknn
entknn(df.to_pandas(), k=2, dtype='euclidean')

-4.738875738790321

In [6]:
from polars_ds.num import query_knn_entropy
df.select(query_knn_entropy('log_return_sp500', 'log_return_vix', k=2))

log_return_sp500
f64
-4.738876


Checking `copula entropy` test from `pycopent` gives similar results as our implemenation 

In [7]:
from copent import copent
copent(df.to_numpy(), k=3, dtype='euclidean')

0.405154203160877

In [8]:
from polars_ds.num import query_copula_entropy
df.select(query_copula_entropy('log_return_vix', 'log_return_sp500', k=3))

log_return_vix
f64
0.405154


Checking `conditional independence` test from `pycopent` gives similar results as our implemenation 

In [9]:
from copent import ci
lag = 2

px1 = df.get_column('log_return_vix').slice(0, len(df) - lag)
px2 = df.get_column('log_return_vix').slice(lag, len(df))
psource = df.get_column('log_return_sp500').slice(0, len(df) - lag)
ci(px2, psource, px1, dtype='euclidean', k=2)

0.08942643134823403

In [10]:
x1 = pl.col.log_return_vix.slice(0, pl.len() - lag)
x2 = pl.col.log_return_vix.slice(lag, pl.len())
source = pl.col.log_return_sp500.slice(0, pl.len() - lag)
df.select(x2.num.conditional_independence(source, x1, k=2))

log_return_vix
f64
0.087541


In [11]:
from copent import transent

transent(df.get_column('log_return_vix'), df.get_column('log_return_sp500'), lag=1, k=2, dtype='euclidean')

0.028790856054837555

In [12]:
df.select(pl.col.log_return_vix.num.transfer_entropy(pl.col('log_return_sp500'), lag=1, k=2))

log_return_vix
f64
0.027849
